## Bull put spread with future

In [ ]:
# On mondays hedge position needs to be moved to next week
#--> to do this currently this below variable is set to 2 because as of today tuesday is the expiry day for nifty

to_pick_next_week = 2

def bull_put_spread_with_future(kite, instruments, lower_premium=40, upper_premium=60):
    print("Printing Bull put spread details")
    import datetime

    nifty_fut_list = [
        inst for inst in instruments
        if inst.get('instrument_type') == 'FUT'
        and inst.get('tradingsymbol', '').startswith("NIFTY")
        and inst.get('name') == 'NIFTY'
        and inst.get('expiry') is not None
    ]

    if not nifty_fut_list:
        print("No NIFTY FUT instruments found.")
        return None, None

    # Pick nearest FUT (first sorted by expiry)
    nifty_fut_list = sorted(nifty_fut_list, key=lambda x: x['expiry'])
    selected_future = nifty_fut_list[0]

    # Fetch LTP
    fut_key = f"{selected_future['exchange']}:{selected_future['tradingsymbol']}"
    future_ltp_data = kite.ltp([fut_key])

    future_price = future_ltp_data[fut_key]['last_price']

    future_symbol = selected_future['tradingsymbol']
    future_token  = selected_future['instrument_token']
    print("")
    print(f"Selected FUT: {future_symbol} (Token: {future_token})  LTP: {future_price}")
    print("")


    # ----------------------------------- #
    #         PUT HEDGE SELECTION
    # ----------------------------------- #
    today = datetime.date.today()
    min_expiry = today + datetime.timedelta(days = to_pick_next_week)

    nifty_pe_list = [
        inst for inst in instruments
        if inst.get('instrument_type') == 'PE'
        and inst.get('tradingsymbol', '').startswith("NIFTY")
        and inst.get('name') == 'NIFTY'
        and inst.get('expiry') is not None
        and inst.get('expiry') > min_expiry
    ]

    if not nifty_pe_list:
        print("No NIFTY PE instruments meeting expiry criteria.")
        return (future_symbol, future_token), None

    # Limit search to reduce API load
    candidate_puts = nifty_pe_list[:200]

    pe_keys = [f"{opt['exchange']}:{opt['tradingsymbol']}" for opt in candidate_puts]
    pe_ltp_data = kite.ltp(pe_keys)

    put_symbol = None
    put_token  = None

    for opt in candidate_puts:
        key = f"{opt['exchange']}:{opt['tradingsymbol']}"
        price = pe_ltp_data[key]['last_price']

        if lower_premium < price < upper_premium:
            put_symbol = opt['tradingsymbol']
            put_token  = opt['instrument_token']
            print("")
            print(f"Selected PUT hedge: {put_symbol} (Token: {put_token}) Premium: {price}")
            print("")
            break

    if put_symbol is None:
        print("No PUT hedge found within premium range.")
        return (future_symbol, future_token), None

    return (future_symbol, future_token), (put_symbol, put_token)


## Bear call spread with future

In [4]:
def bear_call_spread_with_future(kite, instruments, lower_premium=40, upper_premium=60):
    print("Printing Bear call spread details")
    import datetime

    # ----------------------------------- #
    #       FUTURE SELECTION
    # ----------------------------------- #
    nifty_fut_list = [
        inst for inst in instruments
        if inst.get('instrument_type') == 'FUT'
        and inst.get('tradingsymbol', '').startswith("NIFTY")
        and inst.get('name') == 'NIFTY'
        and inst.get('expiry') is not None
    ]

    if not nifty_fut_list:
        print("No NIFTY FUT instruments found.")
        return None, None

    # Sort by expiry and pick nearest FUT
    nifty_fut_list = sorted(nifty_fut_list, key=lambda x: x['expiry'])
    selected_future = nifty_fut_list[0]

    # Fetch LTP
    fut_key = f"{selected_future['exchange']}:{selected_future['tradingsymbol']}"
    future_ltp_data = kite.ltp([fut_key])

    future_price  = future_ltp_data[fut_key]['last_price']
    future_symbol = selected_future['tradingsymbol']
    future_token  = selected_future['instrument_token']
    print("")
    print(f"Selected FUT: {future_symbol} (Token: {future_token})  LTP: {future_price}")
    print("")


    # ----------------------------------- #
    #          CALL HEDGE SELECTION
    # ----------------------------------- #
    today = datetime.date.today()
    min_expiry = today + datetime.timedelta(days  =  to_pick_next_week)

    nifty_ce_list = [
        inst for inst in instruments
        if inst.get('instrument_type') == 'CE'
        and inst.get('tradingsymbol', '').startswith("NIFTY")
        and inst.get('name') == 'NIFTY'
        and inst.get('expiry') is not None
        and inst.get('expiry') > min_expiry
    ]

    if not nifty_ce_list:
        print("No NIFTY CE instruments meeting expiry criteria.")
        return (future_symbol, future_token), None

    candidate_calls = nifty_ce_list[:200]

    ce_keys = [f"{opt['exchange']}:{opt['tradingsymbol']}" for opt in candidate_calls]
    ce_ltp_data = kite.ltp(ce_keys)

    call_symbol = None
    call_token  = None

    for opt in candidate_calls:
        key = f"{opt['exchange']}:{opt['tradingsymbol']}"
        price = ce_ltp_data[key]['last_price']

        if lower_premium < price < upper_premium:
            call_symbol = opt['tradingsymbol']
            call_token  = opt['instrument_token']
            print("")
            print(f"Selected CALL hedge: {call_symbol} (Token: {call_token}) Premium: {price}")
            print("")
            break

    if call_symbol is None:
        print("No CALL hedge found within premium range.")
        return (future_symbol, future_token), None

    return (future_symbol, future_token), (call_symbol, call_token)


In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML

def pick_direction(kite, instruments, lp, up, make_order_basket_fn=None):
    """
    Displays Bullish/Bearish strategy buttons.
    Runs selected strategy and automatically triggers make_order_basket().

    Parameters:
    - kite, instruments: required for user strategies
    - lp, up: premium filters
    - make_order_basket_fn: function that accepts (fut_symbols, option_symbols, kite)

    Returns:
    - fut_symbols, option_symbols, tokens (though now auto-handled)
    """

    output = widgets.Output()

    global fut_symbols, option_symbols, tokens
    fut_symbols = []
    option_symbols = []
    tokens = []

    # ------------------------ BULLISH ------------------------
    def on_bullish_click(b):
        global fut_symbols, option_symbols, tokens

        (fut_symbol, fut_token), (put_symbol, put_token) = bull_put_spread_with_future(
            kite, instruments, lower_premium=lp, upper_premium=up
        )

        fut_symbols = [fut_symbol]
        option_symbols = [put_symbol]
        tokens = [fut_token, put_token]

        with output:
            output.clear_output()
            display(HTML("<h3 style='color:green;'>🟢 BULLISH STRATEGY SELECTED — PRICE EXPECTED TO MOVE UP 📈</h3>"))
            print("▶ FUTURES symbol:", fut_symbols)
            print("▶ OPTION hedge symbol (PUT):", option_symbols)
            print("▶ TOKENS:", tokens)

        # Auto-run make_order_basket
        if make_order_basket_fn:
            make_order_basket_fn(fut_symbols, option_symbols, kite)

    # ------------------------ BEARISH ------------------------
    def on_bearish_click(b):
        global fut_symbols, option_symbols, tokens

        (fut_symbol, fut_token), (call_symbol, call_token) = bear_call_spread_with_future(
            kite, instruments, lower_premium=lp, upper_premium=up
        )

        fut_symbols = [fut_symbol]
        option_symbols = [call_symbol]
        tokens = [fut_token, call_token]

        with output:
            output.clear_output()
            display(HTML("<h3 style='color:red;'>🔴 BEARISH STRATEGY SELECTED — PRICE EXPECTED TO MOVE DOWN 📉</h3>"))
            print("▶ FUTURES symbol:", fut_symbols)
            print("▶ OPTION hedge symbol (CALL):", option_symbols)
            print("▶ TOKENS:", tokens)

        # Auto-run make_order_basket
        if make_order_basket_fn:
            make_order_basket_fn(fut_symbols, option_symbols, kite)

    # ------------------------ BUTTONS ------------------------
    bullish_button = widgets.Button(description="Bullish Strategy", button_style='success')
    bearish_button = widgets.Button(description="Bearish Strategy", button_style='danger')

    bullish_button.on_click(on_bullish_click)
    bearish_button.on_click(on_bearish_click)

    display(widgets.VBox([widgets.HBox([bullish_button, bearish_button]), output]))

    return fut_symbols, option_symbols, tokens


In [ ]:
# import ipywidgets as widgets
# from IPython.display import display, HTML

# def pick_direction(kite, instruments, lp, up):
#     """
#     Displays Bullish/Bearish strategy buttons, runs the selected strategy,
#     and collects futures symbols, option symbols, and tokens.

#     Returns:
#     - fut_symbols: list
#     - option_symbols: list
#     - tokens: list
#     """

#     output = widgets.Output()

#     # Global vars you want to collect
#     global fut_symbols, option_symbols, tokens
#     fut_symbols = []
#     option_symbols = []
#     tokens = []

#     # ------------------------ BULLISH ------------------------
#     def on_bullish_click(b):
#         global fut_symbols, option_symbols, tokens

#         # Your function returns ((symbol, token), (symbol, token))
#         (fut_symbol, fut_token), (put_symbol, put_token) = bull_put_spread_with_future(
#             kite, instruments, lower_premium=lp, upper_premium=up
#         )

#         # Store cleaned values
#         fut_symbols = [fut_symbol]
#         option_symbols = [put_symbol]
#         tokens = [fut_token, put_token]

#         with output:
#             output.clear_output()
#             display(HTML(
#                 "<h3 style='color:green;'>🟢 BULLISH STRATEGY SELECTED — PRICE EXPECTED TO MOVE UP 📈</h3>"
#             ))
#             print("▶ FUTURES symbol:", fut_symbols)
#             print("▶ OPTION hedge symbol (PUT):", option_symbols)
#             print("▶ TOKENS:", tokens)

#     # ------------------------ BEARISH ------------------------
#     def on_bearish_click(b):
#         global fut_symbols, option_symbols, tokens

#         # Your function returns ((symbol, token), (symbol, token))
#         (fut_symbol, fut_token), (call_symbol, call_token) = bear_call_spread_with_future(
#             kite, instruments, lower_premium=lp, upper_premium=up
#         )

#         # Store cleaned values
#         fut_symbols = [fut_symbol]
#         option_symbols = [call_symbol]
#         tokens = [fut_token, call_token]

#         with output:
#             output.clear_output()
#             display(HTML(
#                 "<h3 style='color:red;'>🔴 BEARISH STRATEGY SELECTED — PRICE EXPECTED TO MOVE DOWN 📉</h3>"
#             ))
#             print("▶ FUTURES symbol:", fut_symbols)
#             print("▶ OPTION hedge symbol (CALL):", option_symbols)
#             print("▶ TOKENS:", tokens)

#     # ------------------------ BUTTONS ------------------------
#     bullish_button = widgets.Button(description="Bullish Strategy", button_style='success')
#     bearish_button = widgets.Button(description="Bearish Strategy", button_style='danger')

#     bullish_button.on_click(on_bullish_click)
#     bearish_button.on_click(on_bearish_click)

#     display(widgets.VBox([widgets.HBox([bullish_button, bearish_button]), output]))

#     # Return all 3 variables
#     return fut_symbols, option_symbols, tokens
